Todo: Clean up and rename stuff

In [1]:
import scr_utils as scru
rate_sek=scru.get_sek()

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import pandas as pd
import json
import numpy as np

In [4]:
gameName='Red Dead Redemption 2'
gameNameAlt='Hitman 3'

---- allkeyshop search----- 

In [5]:
url_endpoint = "https://www.allkeyshop.com/blog/wp-admin/admin-ajax.php"
reqParams = {
    'action': 'catalogue_query',
    'query[search]': gameName,
    'query[productsPerPage]': 16,
    'query[currency]': 'eur',
    'query[category]': 'pc-games-all'
}

resp = requests.get(url_endpoint, params=reqParams).json()['results']

In [6]:
soup = BeautifulSoup(resp, 'html.parser')
productId = soup.find(class_="metacritic-button")['data-product-id']
productId

'18688'

In [7]:
reqParams = {
    'action': 'get_offers',
    'product': productId,
    'currency': 'eur',
    'use_beta_offers_display': 1,
    'region':2
}

payload = {}

response = requests.request(
    "GET", url_endpoint, params=reqParams, data=payload)
#response.json()

In [8]:
file=response.json()

In [9]:
file.keys()

dict_keys(['success', 'offers', 'merchants', 'editions', 'regions'])

In [33]:
file['editions']

{'1': {'id': '1', 'name': 'Standard'}, '21': {'id': '21', 'name': 'Ultimate'}}

In [10]:
file['offers'][0]['price']['eur'].keys()

dict_keys(['bestCoupon', 'currency', 'price', 'priceWithoutCoupon'])

In [11]:
file['offers'][0]['price']['eur']['priceWithoutCoupon']

29.99

In [12]:
data=pd.json_normalize(file,record_path='offers')
data

,id,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon,price.eur.currency,price.eur.price,price.eur.priceWithoutCoupon,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback
0,132861340,https://store.steampowered.com/app/1174180/Red...,True,1,1,2,InStock,steam,NaN,eur,29.99,29.99,NaN,NaN,NaN,NaN
1,134728294,https://www.hrkgame.com/en/games/product/red-d...,True,94,21,2,InStock,steam,NaN,eur,33.64,37.38,-10%: ALLKEYSHOP10,10.00,%,False
2,132868941,https://www.cjs-cdkeys.com/products/Red-Dead-R...,True,67,1,2,InStock,steam,NaN,eur,33.87,33.87,NaN,NaN,NaN,NaN
3,134548291,https://store.steampowered.com/sub/412454/,True,1,21,2,InStock,steam,NaN,eur,35.99,35.99,NaN,NaN,NaN,NaN


mapp=data.drop(['price.eur.bestCoupon.discountValue','price.eur.bestCoupon.discountStrategy','price.eur.bestCoupon.isCashback','price.eur.bestCoupon.code', 'price.eur.bestCoupon'], axis=1,inplace=True)

In [13]:
data.rename(columns = {'price.eur.currency':'currency', 'price.eur.price':'price','price.eur.priceWithoutCoupon':'priceWithoutCoupon'}, inplace = True)

In [14]:
data[data.priceWithoutCoupon==data.priceWithoutCoupon.min()]

,id,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon,currency,price,priceWithoutCoupon,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback
0,132861340,https://store.steampowered.com/app/1174180/Red...,True,1,1,2,InStock,steam,NaN,eur,29.99,29.99,NaN,NaN,NaN,NaN


---------- dataframe for merchants ------------

In [15]:
merchants_list=pd.DataFrame(data=file['merchants']).T
merchants_list

,id,name,aggregateRating,types,searchable,paymentMethods,logoSlug,reviewUrl
1,1,Steam,"{'value': 4.8, 'count': 46}",Official Store,1,"{'paymentMethode': ['card', 'paypal']}",steam,https://www.allkeyshop.com/blog/review/steam/
94,94,HRK,"{'value': 2.71, 'count': 324}",CD Key Store,1,"{'paymentMethode': ['card', 'paypal', 'crypto']}",hrk,https://www.allkeyshop.com/blog/review/hrkgame/
67,67,CJS-CDKeys,"{'value': 3.81, 'count': 824}",CD Key Store,1,"{'paymentMethode': ['card', 'paypal']}",cjs-cdkeys,https://www.allkeyshop.com/blog/review/cjs-cdk...


---- merge merchant names with games hits to make it a bit clearer -----

In [16]:
merged_list = pd.merge(data,merchants_list[['id','name']],left_on='merchant',right_on='id',how='left')
merged_list.drop(['id_y','price'], axis=1,inplace=True)

merged_list.rename(columns = {'priceWithoutCoupon':'price'}, inplace = True)
#merged_list.set_index('id_x',inplace=True)

In [17]:
merged_list.head(-12)

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon,currency,price,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,name


In [18]:
instock_list=merged_list.loc[(merged_list['platform'] == 'steam') & (merged_list['stock'] == 'InStock')]

In [19]:
instock_list['price'].describe()

count     4.0000
mean     34.3075
std       3.2199
min      29.9900
25%      32.9000
50%      34.9300
75%      36.3375
max      37.3800
Name: price, dtype: float64

In [20]:
table=instock_list[["name", "price"]].groupby("name").min()
table.sort_values('price', ascending=True,inplace=True)

In [21]:
table.head(5)

,price
name,
Steam,29.99
CJS-CDKeys,33.87
HRK,37.38


In [22]:
instock_list['price'].min()

29.99

In [23]:
instock_list[instock_list.price==instock_list.price.min()]

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon,currency,price,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,name
0,132861340,https://store.steampowered.com/app/1174180/Red...,True,1,1,2,InStock,steam,NaN,eur,29.99,NaN,NaN,NaN,NaN,Steam


In [24]:
instock_list[instock_list.price==instock_list.price.min()]

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon,currency,price,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,name
0,132861340,https://store.steampowered.com/app/1174180/Red...,True,1,1,2,InStock,steam,NaN,eur,29.99,NaN,NaN,NaN,NaN,Steam


In [25]:
allkeysSE = instock_list.assign(priceSEK=instock_list["price"]*rate_sek)
#df2.round({'priceSEK': 1})
allkeysSE.priceSEK=allkeysSE.priceSEK.round(2)
allkeysSE.sort_values(by='priceSEK')

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon,currency,price,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,name,priceSEK
0,132861340,https://store.steampowered.com/app/1174180/Red...,True,1,1,2,InStock,steam,NaN,eur,29.99,NaN,NaN,NaN,NaN,Steam,328.23
2,132868941,https://www.cjs-cdkeys.com/products/Red-Dead-R...,True,67,1,2,InStock,steam,NaN,eur,33.87,NaN,NaN,NaN,NaN,CJS-CDKeys,370.70
3,134548291,https://store.steampowered.com/sub/412454/,True,1,21,2,InStock,steam,NaN,eur,35.99,NaN,NaN,NaN,NaN,Steam,393.90
1,134728294,https://www.hrkgame.com/en/games/product/red-d...,True,94,21,2,InStock,steam,NaN,eur,37.38,-10%: ALLKEYSHOP10,10.00,%,False,HRK,409.12


In [26]:
allkeysSE.loc[allkeysSE['merchant']=='13',['priceSEK','name']].sort_values(by='priceSEK').head(3)

,priceSEK,name


In [27]:
allkeysSE[allkeysSE.priceSEK==allkeysSE.priceSEK.min()]

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon,currency,price,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,name,priceSEK
0,132861340,https://store.steampowered.com/app/1174180/Red...,True,1,1,2,InStock,steam,NaN,eur,29.99,NaN,NaN,NaN,NaN,Steam,328.23


In [28]:
allkeysSE.loc[allkeysSE['priceSEK'].idxmin()]

id_x                                                                             132861340
affiliateUrl                             https://store.steampowered.com/app/1174180/Red...
isActive                                                                              True
merchant                                                                                 1
edition                                                                                  1
region                                                                                   2
stock                                                                              InStock
platform                                                                             steam
price.eur.bestCoupon                                                                   NaN
currency                                                                               eur
price                                                                                29.99

In [29]:
allkeysSE.nsmallest(3,'priceSEK').filter(['name','priceSEK'],axis=1)

,name,priceSEK
0,Steam,328.23
2,CJS-CDKeys,370.70
3,Steam,393.90


In [30]:
allkeysSE.loc[allkeysSE['platform']=='steam',['priceSEK','name']].sort_values(by='priceSEK').head(3)

,priceSEK,name
0,328.23,Steam
2,370.70,CJS-CDKeys
3,393.90,Steam


In [31]:
allkeysSE[allkeysSE.price==allkeysSE.price.min()]

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon,currency,price,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,name,priceSEK
0,132861340,https://store.steampowered.com/app/1174180/Red...,True,1,1,2,InStock,steam,NaN,eur,29.99,NaN,NaN,NaN,NaN,Steam,328.23


In [32]:
scru.to_paraquet('allkey_prices',allkeysSE)

pyarrow.Table
id_x: int64
affiliateUrl: string
isActive: bool
merchant: string
edition: string
region: string
stock: string
platform: string
price.eur.bestCoupon: double
currency: string
price: double
price.eur.bestCoupon.code: string
price.eur.bestCoupon.discountValue: string
price.eur.bestCoupon.discountStrategy: string
price.eur.bestCoupon.isCashback: bool
name: string
priceSEK: double
__index_level_0__: int64
----
id_x: [[132861340,134728294,132868941,134548291]]
affiliateUrl: [["https://store.steampowered.com/app/1174180/Red_Dead_Redemption_2/","https://www.hrkgame.com/en/games/product/red-dead-redemption-2-ultimate-edition-steam-edition","https://www.cjs-cdkeys.com/products/Red-Dead-Redemption-2-CD-Key-For-Steam.html","https://store.steampowered.com/sub/412454/"]]
isActive: [[true,true,true,true]]
merchant: [["1","94","67","1"]]
edition: [["1","21","1","21"]]
region: [["2","2","2","2"]]
stock: [["InStock","InStock","InStock","InStock"]]
platform: [["steam","steam","steam","steam"